In [21]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


def weight_variable(shape):
    init = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init, dtype=tf.float32)
def bias_variable(shape):
    init = tf.constant(0.1, shape = shape)
    return tf.Variable(init, dtype=tf.float32)

#kernel size and strides described over dimensions = [batch_size, h, w, channels]
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
def max_pool_2X2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [22]:
tf.reset_default_graph()
x = tf.placeholder(dtype=tf.float32, shape=[None, 784])
y = tf.placeholder(dtype=tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

#first convolutional layer

#[kwidth, kheight, #inchannels, #outchannels]
W_conv1 = weight_variable([5,5,1,32])
#the same bias is broadcast across all the channels
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])
x_image = tf.cast(x_image, tf.float32)

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2X2(h_conv1)

#second convolutional layer
W_conv2 = weight_variable([5,5,32,64])

b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2X2(h_conv2)

#Fully connected layer, no pooling in these layers
h_pool2 = tf.reshape(h_pool2, [-1, 7*7*64])
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#output layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_predicted = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits = y_predicted)
cross_entropy_loss = tf.reduce_mean(cross_entropy_loss)

train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cross_entropy_loss)

correct_classifications = tf.equal(tf.argmax(y_predicted, 1), tf.argmax(y, 1))
correct_classifications = tf.cast(correct_classifications, tf.float32)
accuracy = tf.reduce_mean(correct_classifications)



#add summary nodes
tf.summary.scalar('Accuracy', accuracy)
tf.summary.scalar('Loss', cross_entropy_loss)
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('Summary/train/')
test_writer = tf.summary.FileWriter('Summary/test/')


In [ ]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    init.run()
    
    for i in range(20000):
        data, labels = mnist.train.next_batch(50)
        summary, _ = sess.run([merged, train_step], feed_dict={x: data, y: labels, keep_prob: 0.5})
        train_writer.add_summary(summary, i)
        
        if i%100 == 0:
            data = mnist.test.images
            labels = mnist.test.labels
            test_summary = sess.run([merged], feed_dict={x:data, y:labels, keep_prob: 1})
            test_writer.add_summary(test_summary, i)
        
            